# Contact Geocode Search

---

***Using Nominatim/OpenStreetMaps to collect the geocodes for contact records that are missing them.***

1. Get list of contacts where latitude == null or longitude == null - include contact ID for posting via API
2. 

In [1]:
import httpx
import json
import numpy as np
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
df = pd.read_excel('../data/raw/Referral_App_Preferred_Providers.xlsx')
df

,Full Name,Person ID,Contact's Work Address,Contact's Work Phone,Contact's Details: Latitude,Contact's Details: Longitude,Contact's Details: Specialty
0,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
1,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
2,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
3,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
4,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
...,...,...,...,...,...,...,...
3069,Waldorf Total Health Chiropractic & Physical T...,996701857,"12102 Old Line Center, Waldorf, MD 20602",(240) 754-7130,38.616663,-76.890752,Chiropractic/Physical Therapy
3070,Waldorf Total Health Chiropractic & Physical T...,996701857,"12102 Old Line Center, Waldorf, MD 20602",(240) 754-7130,38.616663,-76.890752,Chiropractic/Physical Therapy
3071,Washington Circle Orthopaedic Associates,996740755,"3 Washington Circle NW, Ste 404, Washington, D...",(202) 333-2820,NaN,NaN,NaN
3072,Winchester Orthopaedic Associates - Main Locat...,996738975,"128 Medical Circle, Winchester, VA 22601",(540) 667-8975,NaN,NaN,NaN


In [3]:
df = df.rename(columns=lambda x: x.strip())
df

,Full Name,Person ID,Contact's Work Address,Contact's Work Phone,Contact's Details: Latitude,Contact's Details: Longitude,Contact's Details: Specialty
0,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
1,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
2,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
3,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
4,"Absolute Chiropractic Care - Lanham, MD",993709552,"9470 Annapolis Road, Suite 403, Lanham, MD 20706",(301) 577-1800,38.966155,-76.841284,Chiropractic/Physical Therapy
...,...,...,...,...,...,...,...
3069,Waldorf Total Health Chiropractic & Physical T...,996701857,"12102 Old Line Center, Waldorf, MD 20602",(240) 754-7130,38.616663,-76.890752,Chiropractic/Physical Therapy
3070,Waldorf Total Health Chiropractic & Physical T...,996701857,"12102 Old Line Center, Waldorf, MD 20602",(240) 754-7130,38.616663,-76.890752,Chiropractic/Physical Therapy
3071,Washington Circle Orthopaedic Associates,996740755,"3 Washington Circle NW, Ste 404, Washington, D...",(202) 333-2820,NaN,NaN,NaN
3072,Winchester Orthopaedic Associates - Main Locat...,996738975,"128 Medical Circle, Winchester, VA 22601",(540) 667-8975,NaN,NaN,NaN


In [4]:
df = df.drop_duplicates(ignore_index=True)

null_lat = df["Contact's Details: Latitude"].isna()
null_lon = df["Contact's Details: Longitude"].isna()
null_address = df["Contact's Work Address"].notna()

df = (df[null_address & (null_lat | null_lon)]).reset_index(drop=True)
df

,Full Name,Person ID,Contact's Work Address,Contact's Work Phone,Contact's Details: Latitude,Contact's Details: Longitude,Contact's Details: Specialty
0,"Absolute Chiropractic Care - Waldorf, MD",996733740,"3475 Leonardtown Road, Suite 207, Waldorf, MD ...",(301) 835-4422,NaN,NaN,Chiropractic/Physical Therapy
1,"Active Physical Therapy - Annapolis, MD",996733800,"1967-B West St, Annapolis, MD 21401",410-266-3136,NaN,NaN,Chiropractic/Physical Therapy
2,"Active Physical Therapy - Beltsville, MD",996733807,"11000 Baltimore Ave., Ste 107, Beltsville, MD ...",202-331-5460,NaN,NaN,Chiropractic/Physical Therapy
3,Active Physical Therapy - Cambridge,996733823,"309 Sunburst Hwy, Suite #7, Cambridge, MD 21613",410-221-7576,NaN,NaN,Chiropractic/Physical Therapy
4,"Active Physical Therapy - Clinton WHC, MD",996713664,"8887 Woodyard Road, Clinton, MD 20735",301-877-5460,NaN,NaN,Chiropractic/Physical Therapy
...,...,...,...,...,...,...,...
345,The Orthopaedic Foot & Ankle Center - Falls Ch...,996738957,"2922 Telestar Court, Falls Church, VA 22042",(703) 584-2040,NaN,NaN,NaN
346,University Physical Therapy Associates,996748084,"1100 Mercantile Lane, Suite 135, Largo, MD 20774",(301) 322-9495,NaN,NaN,Physical Therapy
347,Washington Circle Orthopaedic Associates,996740755,"3 Washington Circle NW, Ste 404, Washington, D...",(202) 333-2820,NaN,NaN,NaN
348,Winchester Orthopaedic Associates - Main Locat...,996738975,"128 Medical Circle, Winchester, VA 22601",(540) 667-8975,NaN,NaN,NaN


In [5]:
# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="contact_geocode_search")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Create a list to store the geocoded results
geocoded_data = []

# Loop through the dataframe and fetch geocodes for each address
for index, row in df.iterrows():
    try:
        # Fetch the geocode for the contact's work address
        location = geocode(row["Contact's Work Address"])
        if location:
            geocoded_data.append({
                "Full Name": row["Full Name"],
                "Person ID": row["Person ID"],
                "Latitude": location.latitude,
                "Longitude": location.longitude
            })
        else:
            # Append None for missing geocode
            geocoded_data.append({
                "Full Name": row["Full Name"],
                "Person ID": row["Person ID"],
                "Latitude": None,
                "Longitude": None
            })
    except Exception as e:
        print(f"Error geocoding address for {row['Full Name']}: {e}")
        geocoded_data.append({
            "Full Name": row["Full Name"],
            "Person ID": row["Person ID"],
            "Latitude": None,
            "Longitude": None
        })

# Convert the geocoded data into a new dataframe
geocoded_df = pd.DataFrame(geocoded_data)

# Display the resulting dataframe
geocoded_df

RateLimiter caught an error, retrying (0/2 tries). Called with (*('14405 Laurel Place, Suite 102, Laurel, MD 20707',), **{}).
Traceback (most recent call last):
  File "c:\Users\BenjaminMcCarty\miniforge3\envs\streamlit_env\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\BenjaminMcCarty\miniforge3\envs\streamlit_env\Lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BenjaminMcCarty\miniforge3\envs\streamlit_env\Lib\http\client.py", line 1430, in getresponse
    response.begin()
  File "c:\Users\BenjaminMcCarty\miniforge3\envs\streamlit_env\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BenjaminMcCarty\miniforge3\envs\streamlit_env\Lib\http\client.py

KeyboardInterrupt: 

In [8]:
geocoded_df = pd.DataFrame(geocoded_data)
geocoded_df

,Full Name,Person ID,Latitude,Longitude
0,"Absolute Chiropractic Care - Waldorf, MD",996733740,NaN,NaN
1,"Active Physical Therapy - Annapolis, MD",996733800,38.978017,-76.515160
2,"Active Physical Therapy - Beltsville, MD",996733807,NaN,NaN
3,Active Physical Therapy - Cambridge,996733823,NaN,NaN
4,"Active Physical Therapy - Clinton WHC, MD",996713664,38.769374,-76.882966
...,...,...,...,...
284,"Pivot Physical Therapy - RICHMOND, VA (MIDLOTH...",996747531,NaN,NaN
285,"Pivot Physical Therapy - RICHMOND, VA (SCOTT’S...",996747532,37.566148,-77.472909
286,"Pivot Physical Therapy - RICHMOND, VA (WESTHAM...",996747533,37.579260,-77.514811
287,"Pivot Physical Therapy - ROANOKE, VA (TOWERS)",996747534,NaN,NaN


In [10]:
null_lat = geocoded_df['Latitude'].notna()
null_lon = geocoded_df['Longitude'].notna()

geocoded_df = geocoded_df[null_lat & null_lon]
geocoded_df

,Full Name,Person ID,Latitude,Longitude
1,"Active Physical Therapy - Annapolis, MD",996733800,38.978017,-76.515160
4,"Active Physical Therapy - Clinton WHC, MD",996713664,38.769374,-76.882966
10,Active Physical Therapy - Germantown,996733864,39.180165,-77.274085
13,"Active Physical Therapy - Landover, MD",993887356,38.913889,-76.886946
15,"Active Physical Therapy - Oxon Hill, MD",995098756,38.814801,-76.963458
...,...,...,...,...
280,"Pivot Physical Therapy - PRINCE FREDERICK, MD",996747454,38.541872,-76.589252
281,"Pivot Physical Therapy - REISTERSTOWN, MD",996747460,39.461095,-76.827776
285,"Pivot Physical Therapy - RICHMOND, VA (SCOTT’S...",996747532,37.566148,-77.472909
286,"Pivot Physical Therapy - RICHMOND, VA (WESTHAM...",996747533,37.579260,-77.514811
